<a href="https://colab.research.google.com/github/l-ordkp/CelebSearch/blob/master/fine_tuning_whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import re
v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
!pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
!pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2
!pip install librosa soundfile evaluate jiwer torchcodec "datasets>=3.4.1,<4.0.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.5/283.5 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 12.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.11.4 req

In [2]:
import torch
import evaluate
import numpy as np
from unsloth import FastLanguageModel
from transformers import WhisperProcessor, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, Audio
from dataclasses import dataclass
from typing import Any, Dict, List, Union

# --- CONFIGURATION ---
MODEL_ID = "openai/whisper-large-v3"
DATASET_ID = "ai4bharat/IndicVoices"
# 🚩 UPDATED FOR MAITHILI 🚩
DATASET_SUBSET = "maithili"
LANGUAGE_CODE = "mai"    # Maithili ISO 639-2 code is 'mai' (used by Hugging Face)
TASK = "transcribe"

# 1. Load the Processor
# This sets the correct language and task tokens for Maithili transcription.
processor = WhisperProcessor.from_pretrained(
    MODEL_ID, language=LANGUAGE_CODE, task=TASK
)

# Fix for AttributeError: Use the EOS token ID for decoder start.
decoder_start_token_id = processor.tokenizer.eos_token_id

print(f"✅ Processor loaded for {LANGUAGE_CODE.upper()} language.")

/tmp/ipython-input-3267874616.py:4: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

✅ Processor loaded for MAI language.


In [5]:
from google.colab import userdata

In [9]:
from huggingface_hub import notebook_login
# This will open an interactive window where you paste your HF token.
notebook_login()

In [10]:
# 2. Load the IndicVoices Dataset for Maithili
try:
    dataset = load_dataset(
        DATASET_ID,
        DATASET_SUBSET,
        split="train"
    )
    test_dataset = load_dataset(
        DATASET_ID,
        DATASET_SUBSET,
        split="valid"
    )

    # Rename 'text' to 'sentence' and remove unnecessary columns
    dataset = dataset.rename_column("text", "sentence")
    test_dataset = test_dataset.rename_column("text", "sentence")

    columns_to_remove = [col for col in dataset.column_names if col not in ['audio', 'sentence']]
    dataset = dataset.remove_columns(columns_to_remove)
    test_dataset = test_dataset.remove_columns(columns_to_remove)

except Exception as e:
    print(f"FATAL ERROR: Dataset loading failed for {DATASET_ID} ({DATASET_SUBSET}): {e}")
    raise SystemExit(1)

# Cast audio to 16kHz
target_sampling_rate = processor.feature_extractor.sampling_rate
dataset = dataset.cast_column("audio", Audio(sampling_rate=target_sampling_rate))
test_dataset = test_dataset.cast_column("audio", Audio(sampling_rate=target_sampling_rate))

# 3. Preprocessing Function
def prepare_dataset(batch):
    audio = batch["audio"]
    batch["input_features"] = processor.feature_extractor(
        audio["array"], sampling_rate=audio["sampling_rate"]
    ).input_features[0]
    batch["labels"] = processor.tokenizer(batch["sentence"]).input_ids

    # Filter out audio longer than 30 seconds
    if len(audio["array"]) > target_sampling_rate * 30:
        return None

    return batch

# 4. Apply Preprocessing and Filter
dataset = dataset.map(
    prepare_dataset,
    remove_columns=dataset.column_names,
    num_proc=4,
    fn_kwargs={"processor": processor}
).filter(lambda x: x is not None)

test_dataset = test_dataset.map(
    prepare_dataset,
    remove_columns=test_dataset.column_names,
    num_proc=4,
    fn_kwargs={"processor": processor}
).filter(lambda x: x is not None)

print(f"Training Samples (Filtered): {len(dataset)}")
print(f"Test Samples (Filtered): {len(test_dataset)}")

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



FATAL ERROR: Dataset loading failed for ai4bharat/IndicVoices (maithili): Dataset 'ai4bharat/IndicVoices' is a gated dataset on the Hub. Visit the dataset page at https://huggingface.co/datasets/ai4bharat/IndicVoices to ask for access.
Traceback (most recent call last):
  File "/tmp/ipython-input-1368571358.py", line 3, in <cell line: 0>
    dataset = load_dataset(
              ^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/datasets/load.py", line 2062, in load_dataset
    builder_instance = load_dataset_builder(
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/datasets/load.py", line 1782, in load_dataset_builder
    dataset_module = dataset_module_factory(
                     ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/datasets/load.py", line 1652, in dataset_module_factory
    raise e1 from None
  File "/usr/local/lib/python3.12/dist-packages/datasets/load.py", line 1636, in dataset_module_facto

TypeError: object of type 'NoneType' has no len()